# 1. Importações

In [ ]:
import pandas as pd
import numpy as np
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# 2.  Extração do Dataset Kaggle

In [47]:
# Configurações do dataset e diretórios
dataset_slug = 'aungpyaeap/supermarket-sales'
# ATENÇÃO: Confirme se este é o nome EXATO do arquivo dentro do dataset Kaggle.
expected_csv_filename = 'supermarket_sales - Sheet1.csv'
raw_dir = 'csv_raw' # Nome da pasta para armazenar o dado bruto
csv_filepath = os.path.join(raw_dir, expected_csv_filename) # Caminho completo para o arquivo CSV

print(f"Verificando/Preparando para baixar o dataset '{dataset_slug}'...")
print(f"Diretório de destino: '{raw_dir}'")
print(f"Arquivo esperado: '{expected_csv_filename}'")

# Cria o diretório 'csv_raw' se ele não existir
# exist_ok=True evita erro se a pasta já existir
os.makedirs(raw_dir, exist_ok=True)
print(f"Diretório '{raw_dir}' verificado/criado.")

# Verifica se o arquivo CSV já existe na pasta 'csv_raw'
if not os.path.exists(csv_filepath):
    print(f"Arquivo '{expected_csv_filename}' não encontrado em '{raw_dir}'.")
    print(f"Tentando baixar o dataset '{dataset_slug}' do Kaggle para '{raw_dir}'...")

    try:
        # Instancia e autentica a API Kaggle
        api = KaggleApi()
        api.authenticate() # Procura por 'kaggle.json' automaticamente

        # Baixa os arquivos do dataset diretamente para a pasta 'raw_dir' e descompacta
        api.dataset_download_files(dataset_slug, path=raw_dir, unzip=True)

        print(f"Dataset baixado e descompactado em: '{os.path.abspath(raw_dir)}'")

        # Verifica se o arquivo esperado está presente após o download
        if not os.path.exists(csv_filepath):
            files_in_dir = os.listdir(raw_dir)
            print(f"AVISO: O arquivo esperado '{expected_csv_filename}' não foi encontrado após o download.")
            print(f"Arquivos encontrados no diretório '{raw_dir}': {files_in_dir}")
            raise FileNotFoundError(f"Download concluído, mas o arquivo {expected_csv_filename} não está presente em {raw_dir}.")

    except Exception as e:
        print(f"\nErro durante a extração do dataset do Kaggle: {e}")
        print("\nVerifique se:")
        print("1. A biblioteca 'kaggle' está instalada (`pip install kaggle`).")
        print("2. O arquivo 'kaggle.json' está configurado corretamente (geralmente em ~/.kaggle/ ou C:\\Users\\<User>\\.kaggle).")
        print("3. Você aceitou os termos do dataset no site do Kaggle, se necessário.")
        raise SystemExit("Falha na obtenção do arquivo do dataset.")

else:
    print(f"Arquivo '{expected_csv_filename}' já existe em '{raw_dir}'. Download não necessário.")

print(f"\nCaminho final para o arquivo CSV: {csv_filepath}")

Verificando/Preparando para baixar o dataset 'aungpyaeap/supermarket-sales'...
Diretório de destino: 'csv_raw'
Arquivo esperado: 'supermarket_sales - Sheet1.csv'
Diretório 'csv_raw' verificado/criado.
Arquivo 'supermarket_sales - Sheet1.csv' não encontrado em 'csv_raw'.
Tentando baixar o dataset 'aungpyaeap/supermarket-sales' do Kaggle para 'csv_raw'...
Dataset URL: https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales
Dataset baixado e descompactado em: 'c:\Users\cairo\OneDrive\Desktop\test\csv_raw'

Caminho final para o arquivo CSV: csv_raw\supermarket_sales - Sheet1.csv


# 3. Carregamento do DataFrame

In [48]:
# O caminho 'csv_filepath' foi definido no passo 2
# Ele aponta para o arquivo dentro da pasta 'csv_raw'
print(f"\nTentando carregar o DataFrame de: '{csv_filepath}'")

try:
    # Carrega o arquivo CSV para um DataFrame usando o caminho definido
    df = pd.read_csv(csv_filepath)
    print("DataFrame carregado com sucesso.")

except FileNotFoundError:
    print(f"ERRO CRÍTICO: O arquivo CSV esperado não foi encontrado em '{csv_filepath}'.")
    print("Certifique-se de que o PASSO 2 (Extração) foi executado sem erros.")
    raise SystemExit("Erro fatal no carregamento do arquivo.")
except Exception as e:
    print(f"Erro inesperado ao carregar o arquivo CSV '{csv_filepath}': {e}")
    raise SystemExit("Não foi possível carregar o DataFrame.")

# Exibe as primeiras linhas para verificar se carregou corretamente
print("\nDados Carregados (primeiras 5 linhas):")
print(df.head())

# Exibe informações sobre as colunas e tipos de dados
print("\nInformações Iniciais do DataFrame:")
df.info()


Tentando carregar o DataFrame de: 'csv_raw\supermarket_sales - Sheet1.csv'
DataFrame carregado com sucesso.

Dados Carregados (primeiras 5 linhas):
    Invoice ID Branch       City Customer type  Gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             Product line  Unit price  Quantity   Tax 5%     Total       Date  \
0       Health and beauty       74.69         7  26.1415  548.9715   1/5/2019   
1  Electronic accessories       15.28         5   3.8200   80.2200   3/8/2019   
2      Home and lifestyle       46.33         7  16.2155  340.5255   3/3/2019   
3       Health and beauty       58.22         8  23.2880  489.0480  1/27/2019   
4       Sports and travel       86.31         7  30.2085  634.3785   2/8/2019   

    Time      P

# 3. Corrigir a coluna 'Date'

In [49]:
# Verifica o tipo de dado atual da coluna 'Date' (provavelmente 'object')
print(f"\nTipo de dado da coluna 'Date' antes da conversão: {df['Date'].dtype}")

# 1. Converte a coluna 'Date' de string para datetime, especificando o formato original
#    O formato '%m/%d/%Y' indica mês/dia/ano(4 dígitos)
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Verifica o tipo de dado após a conversão para datetime (será datetime64[ns])
print(f"Tipo de dado da coluna 'Date' após conversão para datetime: {df['Date'].dtype}")

# Exibe as primeiras linhas com a coluna 'Date' como objeto datetime
print("\nColuna 'Date' como objeto datetime (primeiras 5 linhas):")
print(df[['Date']].head())

# 2. Formata a coluna 'Date' (que agora é datetime) para o formato string 'DD/MM/YY'
#    '%d' para dia, '%m' para mês, '%y' para ano (2 dígitos)
df['Date'] = df['Date'].dt.strftime('%d/%m/%y')

# Verifica o tipo de dado após a formatação para string (voltará a ser 'object')
print(f"\nTipo de dado da coluna 'Date' após formatação para string DD/MM/YY: {df['Date'].dtype}")

# Exibe as primeiras 5 linhas com o formato de data corrigido
print("\nDados com a coluna 'Date' formatada (primeiras 5 linhas):")
print(df.head())


Tipo de dado da coluna 'Date' antes da conversão: object
Tipo de dado da coluna 'Date' após conversão para datetime: datetime64[ns]

Coluna 'Date' como objeto datetime (primeiras 5 linhas):
        Date
0 2019-01-05
1 2019-03-08
2 2019-03-03
3 2019-01-27
4 2019-02-08

Tipo de dado da coluna 'Date' após formatação para string DD/MM/YY: object

Dados com a coluna 'Date' formatada (primeiras 5 linhas):
    Invoice ID Branch       City Customer type  Gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             Product line  Unit price  Quantity   Tax 5%     Total      Date  \
0       Health and beauty       74.69         7  26.1415  548.9715  05/01/19   
1  Electronic accessories       15.28         5   3.8200   80.2200  08/03/19   
2      

# 4. Limpar Nomes das Colunas

In [50]:
print("\nNomes das colunas originais:")
print(df.columns)

# Cria uma cópia dos nomes das colunas para limpeza
novos_nomes = df.columns.str.strip() # Remove espaços extras no início/fim
novos_nomes = novos_nomes.str.lower() # Converte para minúsculas
novos_nomes = novos_nomes.str.replace(' ', '_', regex=False) # Substitui espaços por underscore
novos_nomes = novos_nomes.str.replace('5%', '5pct', regex=False) # Substitui '5%' por '5pct' para evitar problemas
# Adicione outras substituições se necessário (ex: parênteses, outros caracteres especiais)

# Atribui os novos nomes ao DataFrame
df.columns = novos_nomes

print("\nNomes das colunas após a limpeza:")
print(df.columns)

# Exibe as primeiras linhas com os novos nomes de coluna
print("\nDados com nomes de colunas limpos (primeiras 5 linhas):")
print(df.head())


Nomes das colunas originais:
Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating'],
      dtype='object')

Nomes das colunas após a limpeza:
Index(['invoice_id', 'branch', 'city', 'customer_type', 'gender',
       'product_line', 'unit_price', 'quantity', 'tax_5pct', 'total', 'date',
       'time', 'payment', 'cogs', 'gross_margin_percentage', 'gross_income',
       'rating'],
      dtype='object')

Dados com nomes de colunas limpos (primeiras 5 linhas):
    invoice_id branch       city customer_type  gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

         

# 5. Verificar Tipos de Dados e Valores Nulos

In [51]:
print("\nInformações Finais do DataFrame:")
df.info()

print("\nVerificação de Valores Nulos por Coluna:")
print(df.isnull().sum())


Informações Finais do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   invoice_id               1000 non-null   object 
 1   branch                   1000 non-null   object 
 2   city                     1000 non-null   object 
 3   customer_type            1000 non-null   object 
 4   gender                   1000 non-null   object 
 5   product_line             1000 non-null   object 
 6   unit_price               1000 non-null   float64
 7   quantity                 1000 non-null   int64  
 8   tax_5pct                 1000 non-null   float64
 9   total                    1000 non-null   float64
 10  date                     1000 non-null   object 
 11  time                     1000 non-null   object 
 12  payment                  1000 non-null   object 
 13  cogs                     1000 non-null   floa

# 6. Salvar o DataFrame Limpo em um Novo Arquivo CSV

In [52]:
# Define o nome da nova pasta e o caminho completo do arquivo
pasta_destino = 'csv_cleaned'
caminho_arquivo_limpo = os.path.join(pasta_destino, 'supermarket_sales_cleaned.csv')

# Cria a pasta se ela ainda não existir
os.makedirs(pasta_destino, exist_ok=True)

# Salva o DataFrame no novo arquivo CSV, sem incluir o índice do Pandas
df.to_csv(caminho_arquivo_limpo, index=False, encoding='utf-8')

print(f"\nDataFrame limpo foi salvo com sucesso em: {caminho_arquivo_limpo}")


DataFrame limpo foi salvo com sucesso em: csv_cleaned\supermarket_sales_cleaned.csv


## Análise das Colunas e Relações Esperadas:

**Unit price (Preço Unitário)**: O preço de um único item.  
**Quantity (Quantidade)**: O número de itens comprados.  
**Subtotal (Receita Bruta antes de Impostos)**: Deveria ser Unit price * Quantity. Esta é a base para calcular impostos e o lucro.  
**Tax 5% (Imposto 5%)**: Deveria ser 5% do Subtotal (Subtotal * 0.05).  
**Total**: O valor final pago pelo cliente. Deveria ser Subtotal + Tax 5%.  
**cogs (Cost of Goods Sold - Custo dos Produtos Vendidos)**: O custo que o supermercado teve para adquirir os produtos vendidos. Esta informação geralmente não é calculada a partir do preço de venda; ela é uma informação de custo interno.  
**gross income (Lucro Bruto)**: A diferença entre a receita da venda (Subtotal) e o custo para o supermercado (cogs). Deveria ser Subtotal - cogs.  
**gross margin percentage (Percentual de Margem Bruta)**: O lucro bruto como uma porcentagem da receita (Subtotal). Deveria ser (gross income / Subtotal) * 100.

---
## Investigando os Dados:

Vamos pegar a primeira linha como exemplo (usando os nomes já limpos da etapa anterior, ou os originais se você não os limpou):

- unit_price: 74.69  
- quantity: 7  
- tax_5pct: 26.1415  
- total: 548.9715  
- cogs: 522.83  
- gross_margin_percentage: 4.761904762  
- gross_income: 26.1415  
---

**Cálculo do Subtotal**:  
unit_price * quantity = 74.69 * 7 = 522.83

**Comparação**: Veja que o Subtotal calculado (522.83) é exatamente igual ao valor na coluna cogs (522.83). Isso se repete em outras linhas (ex: linha 2: 15.28 * 5 = 76.4, que é o valor de cogs).

**Conclusão 1**: A coluna cogs neste dataset não representa o Custo dos Produtos Vendidos, mas sim o Subtotal (Receita antes dos Impostos). Isso é um erro comum de nomenclatura em datasets.

**Verificação do Imposto**:  
Calculemos 5% do valor que está na coluna cogs (que acabamos de identificar como Subtotal):  
522.83 * 0.05 = 26.1415.

**Conclusão 2**: O valor na coluna tax_5pct (26.1415) está correto, calculado como 5% do Subtotal (que está erroneamente na coluna cogs).

**Verificação do Total**:  
Calculemos cogs + tax_5pct:  
522.83 + 26.1415 = 548.9715.

**Conclusão 3**: O valor na coluna total (548.9715) está correto, sendo a soma do Subtotal (cogs) e do Imposto (tax_5pct).

**Análise do gross_income**: O valor é 26.1415, que é idêntico ao valor do imposto (tax_5pct).

**Conclusão 4**: A coluna gross_income está erroneamente preenchida com o valor do imposto, e não com o lucro bruto real (Subtotal - Custo Real). Não temos o Custo Real neste dataset.

**Análise do gross_margin_percentage**: O valor é constante (4.7619...). Vamos ver se ele tem relação com o imposto:  
tax_5pct / total * 100 = 26.1415 / 548.9715 * 100 ≈ 4.7619....

**Conclusão 5**: A coluna gross_margin_percentage não representa a margem de lucro, mas sim o percentual que o imposto representa sobre o valor total pago (Tax / Total * 100).

---
## Estratégia de Correção e Validação:

Dado que não temos o Custo Real dos Produtos (COGS verdadeiro), não podemos calcular o gross_income e gross_margin_percentage reais. O melhor a fazer é:

1. Recalcular as colunas cogs (entendendo que é o Subtotal), tax_5pct e total com base nos valores fundamentais unit_price e quantity para garantir consistência e corrigir possíveis erros de cálculo ou arredondamento no arquivo original.

2. Decidir o que fazer com as colunas gross_income e gross_margin_percentage, que estão claramente incorretas ou mal interpretadas no contexto de seus nomes. As opções são:

   - **Removê-las**: É a opção mais segura para evitar confusão no Power BI, já que elas não representam o que seus nomes sugerem.
   - **Mantê-las (com ressalvas)**: Se você quiser manter a informação que elas realmente contêm (valor do imposto e imposto % sobre o total), seria essencial renomeá-las para algo como **tax_amount** e **tax_pct_of_total** ou documentar isso muito bem. Para análise padrão de negócios no Power BI, elas provavelmente não serão úteis com esses nomes.

**Recalcular cogs (como subtotal), tax_5pct e total para garantir a integridade, e remover gross_income e gross_margin_percentage por serem enganosas.**


# 7. Recalcular Colunas Financeiras e Remover Colunas Enganosas

In [53]:
print("\n--- Iniciando Validação e Recálculo Financeiro ---")

# 1. Recalcular o Subtotal (que está na coluna 'cogs')
#    Para garantir consistência, vamos recalcular a partir de unit_price e quantity.
subtotal_calculado = df['unit_price'] * df['quantity']

# Opcional: Comparar o subtotal calculado com a coluna 'cogs' original
# Use np.isclose para comparar números de ponto flutuante (float)
diferencas_cogs = df[~np.isclose(subtotal_calculado, df['cogs'])]
if not diferencas_cogs.empty:
    print("\nALERTA: Encontradas diferenças entre 'cogs' original e 'unit_price * quantity':")
    print(diferencas_cogs[['unit_price', 'quantity', 'cogs']])
else:
    print("\nVerificação: Coluna 'cogs' original corresponde a 'unit_price * quantity'.")

# Sobrescrever/Confirmar a coluna 'cogs' com o cálculo correto do Subtotal
# Arredondar para evitar problemas com muitas casas decimais (ex: 2 casas)
df['cogs'] = (df['unit_price'] * df['quantity']).round(2)
print("Coluna 'cogs' recalculada como Subtotal (unit_price * quantity).")

# 2. Recalcular o Imposto (tax_5pct)
#    Baseado no Subtotal recalculado ('cogs')
df['tax_5pct'] = (df['cogs'] * 0.05).round(2)
print("Coluna 'tax_5pct' recalculada como 5% do Subtotal ('cogs').")

# 3. Recalcular o Total
#    Baseado no Subtotal ('cogs') e Imposto ('tax_5pct') recalculados
df['total'] = (df['cogs'] + df['tax_5pct']).round(2)
print("Coluna 'total' recalculada como Subtotal ('cogs') + Imposto ('tax_5pct').")

# 4. Lidar com as colunas 'gross_income' e 'gross_margin_percentage'
#    Como elas estão incorretas/enganosas, a recomendação é removê-las.
colunas_para_remover = ['gross_income', 'gross_margin_percentage']
# Verifica se as colunas existem antes de tentar remover
colunas_existentes_para_remover = [col for col in colunas_para_remover if col in df.columns]

# 5. Renomeia a coluna 'cogs' para 'subtotal'
df.rename(columns={'cogs': 'subtotal'}, inplace=True)
print("Coluna 'cogs' foi renomeada para 'subtotal'.")

if colunas_existentes_para_remover:
    df = df.drop(columns=colunas_existentes_para_remover)
    print(f"Colunas removidas por serem enganosas: {colunas_existentes_para_remover}")
else:
    print("Colunas 'gross_income' e 'gross_margin_percentage' não encontradas ou já removidas.")

# 5. Verificar o resultado
print("\nDataFrame após recálculo financeiro e remoção de colunas (primeiras 5 linhas):")
print(df.head())

print("\nInformações do DataFrame após ajustes financeiros:")
df.info()


--- Iniciando Validação e Recálculo Financeiro ---

Verificação: Coluna 'cogs' original corresponde a 'unit_price * quantity'.
Coluna 'cogs' recalculada como Subtotal (unit_price * quantity).
Coluna 'tax_5pct' recalculada como 5% do Subtotal ('cogs').
Coluna 'total' recalculada como Subtotal ('cogs') + Imposto ('tax_5pct').
Coluna 'cogs' foi renomeada para 'subtotal'.
Colunas removidas por serem enganosas: ['gross_income', 'gross_margin_percentage']

DataFrame após recálculo financeiro e remoção de colunas (primeiras 5 linhas):
    invoice_id branch       city customer_type  gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             product_line  unit_price  quantity  tax_5pct   total      date  \
0       Health and beauty       74.69

In [54]:
# Define o nome da nova pasta e o caminho completo do arquivo
pasta_destino = 'csv_final'
caminho_arquivo_limpo = os.path.join(pasta_destino, 'supermarket_sales_cleaned_final.csv')

# Cria a pasta se ela ainda não existir
os.makedirs(pasta_destino, exist_ok=True)

# Salva o DataFrame final
df.to_csv(caminho_arquivo_limpo, index=False, encoding='utf-8') # utf-8 é uma boa codificação padrão

print(f"\nDataFrame final foi salvo com sucesso em: {caminho_arquivo_limpo}")


DataFrame final foi salvo com sucesso em: csv_final\supermarket_sales_cleaned_final.csv
